In [8]:
import pandas as pd
from bertopic import BERTopic
import pandas as pd
import spacy
import nltk
import re
import string
import numpy as np
from nltk.corpus import stopwords
#import gensim
#from gensim import corpora
#import matplotlib.pyplot as plt
#import stanza
import spacy
from tqdm import tqdm
import pyprojroot.here as here

In [4]:
from bertopic.backend import BaseEmbedder
from bertopic.cluster import BaseCluster
from bertopic.vectorizers import ClassTfidfTransformer
from bertopic.dimensionality import BaseDimensionalityReduction

# Load Files

In [9]:
# TODO: modify these list if needed (eg. if you want to load only 1 csv from star3, delete other csvs in star3 list)
star3 = ['cleaned_ibis-sg-bencoolen.csv','cleaned_hotel-boss.csv','cleaned_hotel-G.csv',
           'cleaned_village-hotel-albert-court-by-far-east-hospitality.csv',
           'cleaned_holiday-inn-express-clarke-quay.csv']
star4 = ['cleaned_village-hotel-changi-by-far-east-hospitality.csv',
         'cleaned_park-regis.csv', 'cleaned_grand-mercure-sg-roxy.csv',
         'cleaned_paradox-sg-merchant-court.csv','cleaned_crowne-plaza.csv']
star5 = ['cleaned_fullerton.csv', 'cleaned_parkroyal-collection-marina-bay.csv', 'cleaned_pan-pacific.csv',
          'cleaned_mbs_total.csv', 'cleaned_swissotel-the-stamford.csv']

RAW_FOLDER = "data/processed/"

def combine_csv_to_dataframe(file_names, all_star = False, filterDate = True):
    """
    Combine multiple CSV files into a single DataFrame.

    Parameters:
    file_names (list): List of CSV file names. 
    all_star (bool): whether or not to load all the hotels (False if only want to load 1 type of hotel star). 
    filterData (bool): whether or not to remove all data dated before 2015

    Returns:
    pd.DataFrame: Combined DataFrame.
    """
    combined_df = pd.DataFrame()

    for file_name in file_names:
        file_interim_path = RAW_FOLDER + file_name
        file_path = here(file_interim_path)
        try:
            df = pd.read_csv(file_path)
            if all_star:
                if file_name in star3:
                    df["star"] = 3
                elif file_name in star4:
                    df["star"] = 4
                else:
                    df["star"] = 5
            #print(f"Length of {file_name} is {len(df)}")
            combined_df = pd.concat([combined_df, df], ignore_index=True)
            #print(len(combined_df))
        except FileNotFoundError:
            print(f"File not found: {file_name}")
        except pd.errors.EmptyDataError:
            print(f"Empty or invalid CSV file: {file_name}")
            
    combined_df = combined_df[combined_df.year > 2000]
                    
    return combined_df

In [10]:
data = combine_csv_to_dataframe(star3+star4+star5, all_star = True, filterDate = True)
#data[['traveller_username','date','travel_type','traveller_total_contributions','traveller_total_helpful_contributions','review_title','review_text','rating']].head(5)
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68292 entries, 0 to 68291
Data columns (total 19 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   Unnamed: 0                             68292 non-null  int64  
 1   traveller_username                     68292 non-null  object 
 2   review_title                           68253 non-null  object 
 3   review_text                            68292 non-null  object 
 4   travel_type                            31354 non-null  object 
 5   traveller_country_origin               51724 non-null  object 
 6   traveller_total_contributions          68103 non-null  object 
 7   traveller_total_helpful_contributions  54090 non-null  float64
 8   rating                                 54837 non-null  float64
 9   valid_rating                           68292 non-null  bool   
 10  label                                  54837 non-null  object 
 11  cl

,Unnamed: 0,traveller_username,review_title,review_text,travel_type,traveller_country_origin,traveller_total_contributions,traveller_total_helpful_contributions,rating,valid_rating,label,cleaned_review,combined_review,date,covid,year,stem_review,lem_review,star
0,0,Love_Life_Sydney,Clean and comfortable,Hotel rooms in Singapore are so expensive so t...,Trip type: Travelled as a couple,"Sydney, Australia",2302.0,871.0,4.0,True,Positive,clean comfortable hotel rooms singapore expens...,Clean and comfortable Hotel rooms in Singapore...,2023-08-01,PostCovid,2023,clean comfort hotel room singapor expens find ...,clean comfortable hotel room singapore expensi...,3
1,1,Bilal S,"Good hotel, great location",This is a great place! Location is great but t...,Trip type: Travelled with family,"Houston, Texas",4.0,NaN,5.0,True,Positive,good hotel great location great place location...,"Good hotel, great location This is a great pl...",2023-08-01,PostCovid,2023,good hotel great locat great place locat great...,good hotel great location great place location...,3
2,2,Anthony Fernando,Good place for a decent price.,Good place good price Easy access to the city...,Trip type: Travelled with friends,"Dubai, United Arab Emirates",39.0,38.0,5.0,True,Positive,good place decent price good place good price ...,Good place for a decent price. Good place good...,2022-10-01,PostCovid,2022,good place decent price good place good price ...,good place decent price good place good price ...,3
3,3,Mjkc204,Great Location and great staff.,The IBIS was a neat and tidy hotel in line wit...,Trip type: Travelled solo,"Ellenbrook, Australia",37.0,19.0,5.0,True,Positive,great location great staff ibis neat tidy hote...,Great Location and great staff. The IBIS was a...,2023-08-01,PostCovid,2023,great locat great staff ibi neat tidi hotel li...,great location great staff ibis neat tidy hote...,3
4,4,Aung Nanda,Good for budget stay.,I stayed there for 7 days. It was a nice locat...,Trip type: Travelled on business,"Dubai, United Arab Emirates",3.0,4.0,4.0,True,Positive,good budget stay stayed days nice location sev...,Good for budget stay. I stayed there for 7 day...,2022-08-01,PostCovid,2022,good budget stay stay day nice locat seven ele...,good budget stay stay day nice location seven ...,3


# Dependency Parsing

In [12]:
nlp = spacy.load("en_core_web_sm")
data['aspect_sentiment'] = np.nan

In [13]:
## old code
for i in range(len(data)):
    sentence = data['lem_review'][i]
    doc = nlp(sentence)
    aspect_sentiment = []
    for word in doc:
        if word.pos_ == 'NOUN': 
            #print(word)
            for j in word.lefts:
                #print(j, word, j.dep_, j.pos_)
                if j.dep_ == 'amod' and j.pos_ == 'ADJ':
                    tup = (word, j)
                    #print(tup)
                    aspect_sentiment.append(tup)
                for k in j.lefts:
                    if k.dep_ == 'advmod':
                        #print(word, j ,k)
                        tup = (word, k.text + ' ' + j.text)
                        aspect_sentiment.append(tup)
                        if (word, j) in aspect_sentiment:
                            aspect_sentiment.remove((word, j))
    """
        if word.pos_ == 'VERB':
            for j in word.lefts:
                if j.dep_ == 'advmod' and j.pos_ == 'ADV':
                    tup = (word, j)
                    aspect_sentiment.append(tup)
                if j.dep_ == 'neg' and j.pos_ == 'ADV':
                    tup = (word, j)
                    aspect_sentiment.append(tup)
            for j in word.rights:
                if j.dep_ == 'advmod'and j.pos_ == 'ADV':
                    tup = (word, j)
                    aspect_sentiment.append(tup)
    aspect_sentiment
    """
    data['aspect_sentiment'][i] = aspect_sentiment

TypeError: Argument 'other' has incorrect type (expected spacy.tokens.token.Token, got str)

In [23]:
valid_data = data.query('valid_rating==True') #rating is not NaN
len(valid_data)

54837

In [24]:
for i in range(len(valid_data)):
    sentence = valid_data['lem_review'][i]
    doc = nlp(sentence)
    aspect_sentiment = []
    
    for word in doc:
        if word.pos_ == 'NOUN': 
            for j in word.lefts:
                if j.dep_ == 'amod' and j.pos_ == 'ADJ':
                    tup = (word, j)
                    aspect_sentiment.append(tup)
                for k in j.lefts:
                    if k.dep_ == 'advmod':
                        tup = (word, k)  # Changed this line
                        aspect_sentiment.append(tup)
                        if tup in aspect_sentiment:  # Fix this line
                            aspect_sentiment.remove(tup)  # Fix this line

    valid_data['aspect_sentiment'][i] = aspect_sentiment

KeyError: 11902

In [16]:
data['aspect_sentiment']

0        [(room, clean), (room, comfortable), (transpor...
1        [(location, good), (location, great), (place, ...
2        [(place, good), (price, decent), (place, good)...
3        [(location, great), (staff, great), (desk, nea...
4        [(budget, good), (location, nice), (door, next...
                               ...                        
68287    [(swiss, swiss), (time, many), (location, conv...
68288    [(stay, happy), (location, ideal), (view, sens...
68289    [(view, great), (location, great), (room, dece...
68290                    [(date, ordinary), (cover, soft)]
68291    [(book, poor), (year, new), (choice, second), ...
Name: aspect_sentiment, Length: 68292, dtype: object

In [17]:
data.drop(columns=["Unnamed: 0", "traveller_username", "review_title", "review_text", 
                   "traveller_total_contributions", "traveller_total_helpful_contributions",
                  "valid_rating", ""])

,Unnamed: 0,traveller_username,review_title,review_text,travel_type,traveller_country_origin,traveller_total_contributions,traveller_total_helpful_contributions,rating,valid_rating,label,cleaned_review,combined_review,date,covid,year,stem_review,lem_review,star,aspect_sentiment
0,0,Love_Life_Sydney,Clean and comfortable,Hotel rooms in Singapore are so expensive so t...,Trip type: Travelled as a couple,"Sydney, Australia",2302.0,871.0,4.0,True,Positive,clean comfortable hotel rooms singapore expens...,Clean and comfortable Hotel rooms in Singapore...,2023-08-01,PostCovid,2023,clean comfort hotel room singapor expens find ...,clean comfortable hotel room singapore expensi...,3,"[(room, clean), (room, comfortable), (transpor..."
1,1,Bilal S,"Good hotel, great location",This is a great place! Location is great but t...,Trip type: Travelled with family,"Houston, Texas",4.0,NaN,5.0,True,Positive,good hotel great location great place location...,"Good hotel, great location This is a great pl...",2023-08-01,PostCovid,2023,good hotel great locat great place locat great...,good hotel great location great place location...,3,"[(location, good), (location, great), (place, ..."
2,2,Anthony Fernando,Good place for a decent price.,Good place good price Easy access to the city...,Trip type: Travelled with friends,"Dubai, United Arab Emirates",39.0,38.0,5.0,True,Positive,good place decent price good place good price ...,Good place for a decent price. Good place good...,2022-10-01,PostCovid,2022,good place decent price good place good price ...,good place decent price good place good price ...,3,"[(place, good), (price, decent), (place, good)..."
3,3,Mjkc204,Great Location and great staff.,The IBIS was a neat and tidy hotel in line wit...,Trip type: Travelled solo,"Ellenbrook, Australia",37.0,19.0,5.0,True,Positive,great location great staff ibis neat tidy hote...,Great Location and great staff. The IBIS was a...,2023-08-01,PostCovid,2023,great locat great staff ibi neat tidi hotel li...,great location great staff ibis neat tidy hote...,3,"[(location, great), (staff, great), (desk, nea..."
4,4,Aung Nanda,Good for budget stay.,I stayed there for 7 days. It was a nice locat...,Trip type: Travelled on business,"Dubai, United Arab Emirates",3.0,4.0,4.0,True,Positive,good budget stay stayed days nice location sev...,Good for budget stay. I stayed there for 7 day...,2022-08-01,PostCovid,2022,good budget stay stay day nice locat seven ele...,good budget stay stay day nice location seven ...,3,"[(budget, good), (location, nice), (door, next..."
